## Demo: Data Pre-Processing in scikit-learn
---
Author: James D. Triveri


In scikit-learn, data preprocessing refers to the preparation and transformation of raw data into a format that is suitable for machine learning algorithms. This process typically involves several steps, including:

- Handling missing values, outliers, or other inconsistencies in the dataset. Scikit-learn provides tools for imputation of missing values.

- Scaling numerical features to ensure that they have similar scales. This is important for many machine learning algorithms, particularly those based on distance calculations or gradient descent optimization. Scikit-learn offers various scaling methods such as standardization (StandardScaler) and min-max scaling (MinMaxScaler).

- Converting categorical features into a numerical format suitable for machine learning algorithms. This may involve techniques such as one-hot encoding (`OneHotEncoder`) or label encoding (`LabelEncoder`).



## Creating Train-Test Splits

The primary purpose of splitting data into train and test sets is to evaluate the performance of the model on unseen data. By training the model on a subset of the data (the training set) and then evaluating its performance on a separate subset (the test set), we can assess how well the model generalizes to new, unseen data. This helps us understand whether the model has learned meaningful patterns from the data or if it"s simply memorizing the training examples, which is the definition of overfitting. Overfitting occurs when a model learns to perform well on the training data but fails to generalize to new data. By evaluating the model on a separate test set, we can detect overfitting. If the model performs significantly worse on the test set compared to the training set, it indicates that the model has overfit the training data, and adjustments may be needed to improve its generalization ability.

In addition, machine learning models often have hyperparameters that need to be tuned for optimal performance. Splitting the data allows us to perform hyperparameter tuning on the training set while keeping the test set untouched. This helps prevent "leakage" of information from the test set into the model training process, which could lead to overly optimistic performance estimates.


### Preparing the Small Ames Housing Dataset for a Regression Task

The Small Ames Housing Dataset consists of 2,930 records, each of which represent homes sales in Ames. 
The target variable is `sale_price`. The goal would be to create a model to predict `sale_price` using 
the other columns in the dataset. 

We read the data into a Pandas DataFrame and display the first few records:

In [1]:

import numpy as np
import pandas as pd


df = pd.read_csv(
    "https://gist.githubusercontent.com/jtrive84/9b96df3f5b23ef1cef68c6bbe5983153/raw/588858f552167732de4d2440505eab54d8d80316/ames-housing-small.csv",
    na_values=""
    )

df.head()

,lot_area,bld_type,house_style,exterior,foundation,basement_sqft,central_air,first_floor_sqft,full_bath,kitchen_score,fireplaces,garage_type,garage_nbr_cars,paved_drive,sale_price
0,31770,1Fam,1Story,BrkFace,CBlock,1080.0,Y,1656,1,TA,2,Attchd,2,P,215000
1,11622,1Fam,1Story,VinylSd,CBlock,882.0,Y,896,1,TA,0,Attchd,1,Y,105000
2,14267,1Fam,1Story,Wd Sdng,CBlock,1329.0,Y,1329,1,Gd,0,Attchd,1,Y,172000
3,11160,1Fam,1Story,BrkFace,CBlock,2110.0,Y,2110,2,Ex,2,Attchd,2,Y,244000
4,13830,1Fam,2Story,VinylSd,PConc,928.0,Y,928,2,TA,1,Attchd,2,Y,189900



The first step is to identify which columns represent continuous features and which represent categorical features. Categorical features are usually strings (or objects), but can also be integer-valued (for example, `fireplaces` should be treated as a categorical feature, not a continuous feature, since the column only takes on 5 distinct values). 

Note also that categorical features can be one of two types:

- **Nominal features** are categorical variables where the categories do not have a natural ordering or hierarchy ("bird", "cat", "dog").

- **Ordinal features** are categorical variables where the categories have a natural order or hierarchy ("low", "medium", "high"), or "fireplaces" from our dataset. 

We can print the data types of each column in the DataFrame to give us an idea of which features are continuous and which are categorical:

In [2]:
df.dtypes

lot_area              int64
bld_type             object
house_style          object
exterior             object
foundation           object
basement_sqft       float64
central_air          object
first_floor_sqft      int64
full_bath             int64
kitchen_score        object
fireplaces            int64
garage_type          object
garage_nbr_cars       int64
paved_drive          object
sale_price            int64
dtype: object

We treat `lot_area`, `first_floor_sqft` and `basement_sqft` as continuous features. All other columns will be considered categorical. 

In [3]:

target = "sale_price"

continuous = [
    "lot_area", "first_floor_sqft", "basement_sqft"
    ]

categorical = [
    "bld_type", "house_style", "exterior", "foundation", "central_air", 
    "full_bath", "kitchen_score", "fireplaces", "garage_type", "garage_nbr_cars", 
    "paved_drive"
    ]


scikit-learn has good support for imputing continuous features, but for categorical features we will typically forgo imputation. Instead of imputing missing categorical values, we identify them as "missing", and treat those values as a category unto themselves. This will allow us to assess whether these missing values have any association with the target variable as a group. 

In [4]:

# Set any missing categorical features to "missing".
for feature in categorical:
    df[feature] = df[feature].astype("str").fillna("missing")


# Strip whitespace from object columns.
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


We are now in a position to create our preprocessing pipelines. We initialize a `ColumnTransformer` instance, which gives us the ability to define separate preprocessing steps for different groups of columns (in our case, categorical vs. continuous). 

Let's assume we intend to use the `LinearRegression` model in scikit-learn. As most models don't support categorical features, it is necessary to encode them using a numerical representation. We will [one-hot encode](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) our categorical features within our pre-processing pipeline. 

Note that some models can accept ordinal categorical features directly (for example, `GradientBoostingRegressor`), but for now we are assuming all categorical features will be one-hot encoded. 

We also impute missing continuous values using `KNNImputer`. This is preferable to `SimpleImputer` since it doesn't assume independence of features as `SimpleImputer` does. 

Finally, we standardize the continuous features to have mean 0 and standard deviation 1. This is not strictly necessary, but helps convergence for models that rely on gradient descent. It also makes model coefficients more easily interpretable. 

Here is how we would setup the pre-processing pipeline, starting with creating the train-validation-test split. We create three separate datasets, which requires calling `train_test_split` twice. 



In [5]:
"""
Example pre-processing pipeline for Small Ames Housing Dataset.
"""
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


# Create train-validation-test split.
y = df[target]
dftrain, dfvalid, ytrain, yvalid = train_test_split(df, y, test_size=.35, random_state=516)
dfvalid, dftest, yvalid, ytest = train_test_split(dfvalid, yvalid, test_size=.35, random_state=516)


# Continuous feature preprocessing  pipeline.
continuous_transformer = make_pipeline(
    KNNImputer(), 
    StandardScaler()
    )

# Categorical feature preprocessing pipeline.
categorical_transformer = make_pipeline(
    OneHotEncoder(drop="first", sparse_output=False)
    )

# Combine categorical and continuous feature pipelines.
transformer = ColumnTransformer(transformers=[
    ("continuous" , continuous_transformer, continuous),  
    ("categorical", categorical_transformer, categorical)
    ], remainder="drop", verbose_feature_names_out=False
    ).set_output(transform="pandas")


# Call fit_transform on dftrain.
dftrain = transformer.fit_transform(dftrain)


# Only call transform on dfvalid and dftest. 
dfvalid = transformer.transform(dfvalid)
dftest = transformer.transform(dftest)


print(f"dftrain.shape: {dftrain.shape}")
print(f"dfvalid.shape: {dfvalid.shape}")
print(f"dftest.shape : {dftest.shape}")

dftrain.shape: (1902, 48)
dfvalid.shape: (666, 48)
dftest.shape : (359, 48)


Note that we call `fit_transform` on the training set but only `transform` on the test set. This is to prevent leaking information from our test set into our training set. This is a common mistake that even experienced machine learning practitioners sometimes fall victim to. **Ensure `fit_transform` is only ever called on the training set, and that only `transform` is called on the test/validation set.**.

We can inspect our training data and see that the specified preprocessing steps have been executed:


In [6]:
dftrain.head()

,lot_area,first_floor_sqft,basement_sqft,bld_type_2fmCon,bld_type_Duplex,bld_type_Twnhs,bld_type_TwnhsE,house_style_1.5Unf,house_style_1Story,house_style_2.5Fin,...,garage_type_BuiltIn,garage_type_CarPort,garage_type_Detchd,garage_type_nan,garage_nbr_cars_1,garage_nbr_cars_2,garage_nbr_cars_3,garage_nbr_cars_4,paved_drive_P,paved_drive_Y
1509,0.778125,-0.729801,-0.537453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1439,0.328971,0.860988,1.026819,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1285,-0.114787,-0.535428,-0.620512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1375,0.136705,-0.210620,-0.156767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1903,-0.606366,-0.893483,-0.555911,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Continuous feature names are prefixed with `continuous_` and categorical feature names with `categorical_`. Each categorical column will now consist only of 1s and 0s. Notice there are no longer string or object type columns:

In [7]:

dftrain.dtypes


lot_area               float64
first_floor_sqft       float64
basement_sqft          float64
bld_type_2fmCon        float64
bld_type_Duplex        float64
bld_type_Twnhs         float64
bld_type_TwnhsE        float64
house_style_1.5Unf     float64
house_style_1Story     float64
house_style_2.5Fin     float64
house_style_2.5Unf     float64
house_style_2Story     float64
house_style_SFoyer     float64
house_style_SLvl       float64
exterior_CemntBd       float64
exterior_HdBoard       float64
exterior_MetalSd       float64
exterior_Plywood       float64
exterior_VinylSd       float64
exterior_Wd Sdng       float64
exterior_WdShing       float64
exterior_nan           float64
foundation_CBlock      float64
foundation_PConc       float64
foundation_nan         float64
central_air_Y          float64
full_bath_1            float64
full_bath_2            float64
full_bath_3            float64
full_bath_4            float64
kitchen_score_Fa       float64
kitchen_score_Gd       float64
kitchen_


Finally, we are ready to use our data to fit a model. Let's use the `LinearRegression` model mentioned earlier:

In [8]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lr = LinearRegression()

# Fit model on training set. 
mdl = lr.fit(dftrain, ytrain)

# Training predictions. 
ypred_train = mdl.predict(dftrain)

# Generate predictions on validation  set.
ypred_valid = mdl.predict(dfvalid)

# Assess model on final test set. The test set is typically used to compare 
# the output of many models to see how well they perform on unseen data.
ypred_test = mdl.predict(dftest)


# Compute mean absolute error.
train_mae = mean_absolute_error(ytrain, ypred_train)
valid_mae = mean_absolute_error(yvalid, ypred_valid)
test_mae = mean_absolute_error(ytest, ypred_test)

print(f"Train MAE     : {train_mae:,.2f}.")
print(f"Validation MAE: {valid_mae:,.2f}.")
print(f"Test MAE      : {test_mae:,.2f}.")



Train MAE     : 22,404.13.
Validation MAE: 24,118.99.
Test MAE      : 20,605.65.
